In [1]:
from cassandra.cluster import Cluster
cluster = Cluster(["demo-db-1", "demo-db-2", "demo-db-3"])
cass = cluster.connect()

In [3]:
cass.execute("use banking")

In [14]:
cass.execute("drop table if exists loans")

In [15]:
cass.execute("""
create table loans(
    bank_id INT,
    bank_name TEXT static,
    loan_id UUID,
    amount INT,
    state TEXT,
    PRIMARY KEY ((bank_id), amount, loan_id)
) WITH CLUSTERING ORDER BY (amount DESC)
""")

In [16]:
print(cass.execute("describe table loans").one().create_statement)

CREATE TABLE banking.loans (
    bank_id int,
    amount int,
    loan_id uuid,
    bank_name text static,
    state text,
    PRIMARY KEY (bank_id, amount, loan_id)
) WITH CLUSTERING ORDER BY (amount DESC, loan_id ASC)
    AND additional_write_policy = '99p'
    AND bloom_filter_fp_chance = 0.01
    AND caching = {'keys': 'ALL', 'rows_per_partition': 'NONE'}
    AND cdc = false
    AND comment = ''
    AND compaction = {'class': 'org.apache.cassandra.db.compaction.SizeTieredCompactionStrategy', 'max_threshold': '32', 'min_threshold': '4'}
    AND compression = {'chunk_length_in_kb': '16', 'class': 'org.apache.cassandra.io.compress.LZ4Compressor'}
    AND memtable = 'default'
    AND crc_check_chance = 1.0
    AND default_time_to_live = 0
    AND extensions = {}
    AND gc_grace_seconds = 864000
    AND max_index_interval = 2048
    AND memtable_flush_period_in_ms = 0
    AND min_index_interval = 128
    AND read_repair = 'BLOCKING'
    AND speculative_retry = '99p';


# Insert Data

In [17]:
cass.execute("""
INSERT INTO loans (bank_id, bank_name)
VALUES (544, 'test')
""")

In [19]:
import pandas as pd
pd.DataFrame(cass.execute("select * from loans"))

,bank_id,amount,loan_id,bank_name,state
0,544,None,None,test,None


In [20]:
# INSERT is really an UPSERT (UPDATE or INSERT)
cass.execute("""
INSERT INTO loans (bank_id, bank_name)
VALUES (544, 'test2')
""")

In [21]:
import pandas as pd
pd.DataFrame(cass.execute("select * from loans"))

,bank_id,amount,loan_id,bank_name,state
0,544,None,None,test2,None


In [23]:
# cass.execute("""
# INSERT INTO loans (bank_id, bank_name, loan_id)
# VALUES (544, 'test2', UUID())
# """)

In [24]:
# INSERT is really an UPSERT (UPDATE or INSERT)
cass.execute("""
INSERT INTO loans (bank_id, amount, loan_id)
VALUES (544, 300, UUID())
""")

In [25]:
pd.DataFrame(cass.execute("select * from loans"))

,bank_id,amount,loan_id,bank_name,state
0,544,300,b53eb536-1fc6-4add-899a-f159e6b1b052,test2,None


In [26]:
# NOW() and UUID() both return UUIDs
# NOW() is "more" unique (looks at MAC address, timestamp, sequence number)

In [28]:
cass.execute("""
INSERT INTO loans (bank_id, bank_name, amount, loan_id, state)
VALUES (544, 'mybank', 400, NOW(), 'wi')
""")

In [29]:
pd.DataFrame(cass.execute("select * from loans"))

,bank_id,amount,loan_id,bank_name,state
0,544,400,847c6480-7cdf-11ee-aa37-6b2682dad796,mybank,wi
1,544,300,b53eb536-1fc6-4add-899a-f159e6b1b052,mybank,None


In [30]:
cass.execute("""
INSERT INTO loans (bank_id, bank_name, amount, loan_id, state)
VALUES (999, 'uwcu', 500, NOW(), 'il')
""")

In [31]:
pd.DataFrame(cass.execute("select * from loans"))

,bank_id,amount,loan_id,bank_name,state
0,544,400,847c6480-7cdf-11ee-aa37-6b2682dad796,mybank,wi
1,544,300,b53eb536-1fc6-4add-899a-f159e6b1b052,mybank,None
2,999,500,f29d9970-7cdf-11ee-8c42-714f42f16ef6,uwcu,il


In [32]:
cass.execute("""
CREATE TYPE FullName (first text, last text)
""")

In [33]:
cass.execute("""
alter table loans add (username FullName)
""")

In [35]:
cass.execute("""
INSERT INTO loans (bank_id, bank_name, amount, loan_id, username)
VALUES (999, 'uwcu', 500, NOW(), {first:'Tyler', last:'Caraza-Harter'})
""")

In [37]:
pd.DataFrame(cass.execute("select username, username.first, username.last from loans"))

,username,username_first,username_last
0,None,None,None
1,None,None,None
2,None,None,None
3,"(Tyler, Caraza-Harter)",Tyler,Caraza-Harter
